#### For Colab

In [1]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [2]:
!nvidia-smi

Thu Mar 11 09:31:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [5]:
!cd '/content/drive/My Drive/MAGMA: Summarization/transformers/'; pip install -q -e .
!pip install -q -U sentence-transformers
!pip install -q -U wandb
!pip install -q -U rouge-score sacrebleu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 21.4MB/s 
     |████████████████████████████████| 3.2MB 50.5MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 1.2MB 24.1MB/s 
     |████████████████████████████████| 2.0MB 19.0MB/s 
     |████████████████████████████████| 133kB 57.1MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 163kB 51.9MB/s 
     |████████████████████████████████| 71kB 12.2MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 


In [6]:
print("Setting up colab environment")
!pip uninstall -y -q pyarrow
!pip install -q -U ray[tune]
!pip install -q ray[debug]

Setting up colab environment
     |████████████████████████████████| 47.5MB 65kB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 1.3MB 53.7MB/s 
     |████████████████████████████████| 81kB 13.3MB/s 
     |████████████████████████████████| 133kB 63.0MB/s 
     |████████████████████████████████| 3.2MB 57.1MB/s 
     |████████████████████████████████| 204kB 46.2MB/s 
     |████████████████████████████████| 317kB 45.4MB/s 
     |████████████████████████████████| 71kB 12.4MB/s 
     |████████████████████████████████| 296kB 56.5MB/s 
     |████████████████████████████████| 143kB 61.7MB/s 


In [7]:
import sys

print("\n##### sys.path original #####")
for p in sys.path:
    print(f"'{p}'")
print()

import site
site.main()

print("\n##### sys.path after site.main() #####")
for p in sys.path:
    print(f"'{p}'")
print()


##### sys.path original #####
''
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'


##### sys.path after site.main() #####
'/content'
'/env/python'
'/usr/lib/python37.zip'
'/usr/lib/python3.7'
'/usr/lib/python3.7/lib-dynload'
'/usr/local/lib/python3.7/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.7/dist-packages/IPython/extensions'
'/root/.ipython'
'/content/drive/My Drive/MAGMA: Summarization/transformers/src'



### **Config**

In [8]:
import sys
sys.path.insert(0, drive_dir)
import config

import wandb
wandb.login()

project_name = 'ft_t5_bull_para_embed'
%env WANDB_PROJECT=$project_name

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=ft_t5_para_wordembed


In [9]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'

In [10]:
model_name_or_path = 't5-large'

In [11]:
data_dir = '"'+drive_dir+'datasets/bullet_paragraph_embeddings/t5/st/base/"'

source_val_dir = data_dir[:-1] + 'val.source"'
reference_val_dir = data_dir[:-1] + 'val.target"'

source_test_dir = data_dir[:-1] + 'test.source"'
reference_test_dir = data_dir[:-1] + 'test.target"'

In [12]:
GAS = 16
LR = 5e-5

output_dir = '"'+drive_dir+'fine-tuning/'+project_name+'_gas'+\
    str(GAS)+'_lr' + str(LR) + '"'

### Evaluation Out of the Box

In [ ]:
eval_model_name_or_path = '"t5-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_val.json"'
print(save_dir)

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_t5_para_wordembed_gas16_lr5e-05/t5-large_para_wordembed_val.txt"
100% 306M/306M [00:17<00:00, 17.3MB/s]
parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
Downloading: 100% 1.20k/1.20k [00:00<00:00, 1.41MB/s]
Downloading: 100% 2.95G/2.95G [00:44<00:00, 66.0MB/s]
Downloading: 100% 792k/792k [00:00<00:00, 2.45MB/s]
Downloading: 100% 1.39M/1.39M [00:00<00:00, 3.60MB/s]
  0% 0/34 [00:00<?, ?it/s]2021-03-10 14:37:39.541384: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [04:58<00:00,  8.78s/it]
{'rouge1_precision': 25.39, 'rouge1_recall': 43.07, 'rouge1_fmeasure': 30.520000000000003, 'rouge2_precision': 9.9, 'rouge2_recall': 18.17, 'rouge2_fmeasure': 12.19, 'rougeL_precision': 18.73, 'rougeL_recall': 32.71, 'rougeL_fmeasure': 22.78, 'rougeLsum_precision': 21.59, 'rougeLsum_recall': 36.94, 'rougeLsum_fmeasure': 26.02, 'sentenc

In [ ]:
eval_model_name_or_path = '"t5-large"'

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/31 [00:00<?, ?it/s]2021-03-10 14:44:58.318919: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 31/31 [04:29<00:00,  8.70s/it]
{'rouge1_precision': 21.75, 'rouge1_recall': 41.71, 'rouge1_fmeasure': 27.32, 'rouge2_precision': 7.1499999999999995, 'rouge2_recall': 14.38, 'rouge2_fmeasure': 9.11, 'rougeL_precision': 15.89, 'rougeL_recall': 31.28, 'rougeL_fmeasure': 20.11, 'rougeLsum_precision': 18.21, 'rougeLsum_recall': 35.260000000000005, 'rougeLsum_fmeasure': 22.95, 'sentence_distilroberta_cosine': 53.14320921897888, 'w2v_cosine': 99.99673366546631, 'n_obs': 242, 'runtime': 269, 'seconds_per_sample': 1.1116, 'min_length': 10, 'max_length': 150}


### Fine-Tuning

In [ ]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['num_beams'] = 2
model_config_dir = '"'+drive_dir+'fine-tuning/config/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--freeze_embeds \
--adafactor --learning_rate $LR --lr_scheduler constant \
--do_train \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--sortish_sampler \
--task summarization \
--max_source_length 512 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--max_steps 700 \
--logging_steps 7 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--gradient_accumulation_steps $GAS \
--evaluation_strategy steps --eval_steps 7 --eval_beams 2 \
--predict_with_generate \
--save_steps 49 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

Streaming output truncated to the last 5000 lines.
100%|██████████| 34/34 [02:56<00:00,  4.13s/it]

Batches: 100%|██████████| 1/1 [00:00<00:00, 122.05it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 156.57it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 161.31it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 125.95it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 156.80it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 161.91it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 168.66it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 165.36it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 155.80it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 160.55it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 164.34it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 163.80it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 148.42it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 155.14it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 157.61it/s]


Batches: 100%|██████████| 1/1 

### Evaluation

In [13]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-441"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_val.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_val.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_val_dir \
$save_dir \
--reference_path $reference_val_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_t5_para_wordembed_gas16_lr5e-05/checkpoint-441"

100% 306M/306M [00:13<00:00, 21.9MB/s]
parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/34 [00:00<?, ?it/s]2021-03-11 09:36:39.204653: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 34/34 [02:28<00:00,  4.35s/it]
{'rouge1_precision': 36.97, 'rouge1_recall': 32.59, 'rouge1_fmeasure': 33.26, 'rouge2_precision': 15.690000000000001, 'rouge2_recall': 13.819999999999999, 'rouge2_fmeasure': 14.13, 'rougeL_precision': 29.17, 'rougeL_recall': 25.88, 'rougeL_fmeasure': 26.279999999999998, 'rougeLsum_precision': 29.759999999999998, 'rougeLsum_recall': 26.369999999999997, 'rougeLsum_fmeasure': 26.86, 'sentence_distilroberta_cosine': 60.788363218307495, 'w2v_cosine': 99.99619126319885, 'n_obs': 267, 'runtime': 148, 'seconds_per_sample': 0.5543, 'min_length': 10, 'max_length': 150}


In [14]:
eval_model_name_or_path = output_dir[:-1] + '/checkpoint-441"'
print(eval_model_name_or_path+'\n')

save_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_test.txt"'
score_dir = output_dir[:-1] + '/'+eval_model_name_or_path[1:-1].replace('/', '?')+\
    '_bull_para_embed_test.json"'

!python3 $eval_script \
$eval_model_name_or_path \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--bs 8 \
--dump-args \
--min_length $config.ONE_BULLET_MIN_LEN \
--max_length $config.ONE_BULLET_MAX_LEN

"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/ft_t5_para_wordembed_gas16_lr5e-05/checkpoint-441"

parsed the following generate kwargs: {'min_length': 10, 'max_length': 150}
  0% 0/31 [00:00<?, ?it/s]2021-03-11 09:41:19.637343: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 31/31 [02:08<00:00,  4.14s/it]
{'rouge1_precision': 33.660000000000004, 'rouge1_recall': 33.629999999999995, 'rouge1_fmeasure': 31.91, 'rouge2_precision': 14.13, 'rouge2_recall': 13.639999999999999, 'rouge2_fmeasure': 13.16, 'rougeL_precision': 27.439999999999998, 'rougeL_recall': 27.339999999999996, 'rougeL_fmeasure': 25.97, 'rougeLsum_precision': 28.07, 'rougeLsum_recall': 27.91, 'rougeLsum_fmeasure': 26.52, 'sentence_distilroberta_cosine': 58.97238254547119, 'w2v_cosine': 99.99504685401917, 'n_obs': 242, 'runtime': 128, 'seconds_per_sample': 0.5289, 'min_length': 10, 'max_length': 150}
